In [1]:
import pandas as pd
import  matplotlib.pyplot as plt
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
df = pd.read_csv('english_cleaned_lyrics.csv')

In [3]:
df.drop(columns='Unnamed: 0', inplace = True)
df.drop(columns='index', inplace = True)

In [4]:
display(df)

,song,year,artist,genre,lyrics
0,ego-remix,2009,beyonce-knowles,Pop,Oh baby how you doing You know I'm gonna cut r...
1,then-tell-me,2009,beyonce-knowles,Pop,playin everything so easy it's like you seem s...
2,honesty,2009,beyonce-knowles,Pop,If you search For tenderness It isn't hard to ...
3,you-are-my-rock,2009,beyonce-knowles,Pop,Oh oh oh I oh oh oh I If I wrote a book about ...
4,black-culture,2009,beyonce-knowles,Pop,Party the people the people the party it's pop...
...,...,...,...,...,...
218205,who-am-i-drinking-tonight,2012,edens-edge,Country,I gotta say Boy after only just a couple of da...
218206,liar,2012,edens-edge,Country,I helped you find her diamond ring You made me...
218207,last-supper,2012,edens-edge,Country,Look at the couple in the corner booth Looks a...
218208,christ-alone-live-in-studio,2012,edens-edge,Country,When I fly off this mortal earth And I'm measu...


In [5]:
display(df.isnull().sum())
df[df.isnull().any(axis=1)]

song      1
year      0
artist    0
genre     0
lyrics    0
dtype: int64

,song,year,artist,genre,lyrics
116724,NaN,2009,booker-t-and-the-mg-s,Jazz,All right people the rest of the hard working ...


Un seul NaN dans tout le tableau, on peut le drop sans aucun problème car nous avons beaucoup de données.

In [6]:
df = df.drop(index= 116724)

In [7]:
display(df.value_counts(df['genre']))

genre
Rock          100053
Pop            34137
Hip-Hop        22654
Metal          21210
Country        14158
Jazz            7309
Electronic      6942
Other           3786
R&B             3336
Indie           2935
Folk            1689
dtype: int64

Le nombre de chansons des différents genres varie beaucoup, quelques uns en ont beaucoup plus que d'autres, particulièrement Rock.
Si on entraîne un modèle sur ces données, on risquerait d'avoir un gros biais pour ces genres. On devrait donc rendre le set un peu plus 'équilibré'. Pour que ce soit réellement équilibré on doit perdre beaucoup d'entrées, mais j'espère que 1500 par genre reste assez.

In [8]:
df_sample = df.groupby('genre').sample(1500, random_state = 42)
#tout en minuscules
df_sample['lyrics'].str.lower()

51370     a little bitty of croonin a little bitty of sp...
112608    a little song a little dance a little seltzer ...
196290    if i wake up in the morning when i look outsid...
117823    yes she's my lady luck hey i'm her wild card m...
207724    candy kisses wrapped in paper mean more to you...
                                ...                        
34831     i'm not sure is it how you would say yes your ...
40411     everybody everybody said hello every reason ev...
214411    i've sharpened all my tools i follow all the r...
96571     oh i ain't joking when i'm telling ya that whi...
203366    where i come from silvery trees diamond on lea...
Name: lyrics, Length: 16500, dtype: object

In [9]:
stopwords = nltk.corpus.stopwords.words('english')

df_sample['tokenized'] = df_sample.apply(lambda row: nltk.word_tokenize(row['lyrics']), axis=1)

In [10]:
df_sample['tokenized'] = df_sample['tokenized'].apply(lambda x : [word for word in x if word not in stopwords])


In [14]:
v = TfidfVectorizer()
x = v.fit_transform(df_sample['lyrics'])


scipy.sparse.csr.csr_matrix

(16500, 51315)